In [141]:
import os
import json
import numpy as np

from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.data.rmg import RMGDatabase
from rmgpy.species import Species
from rmgpy import settings
from rmgpy.kinetics import Arrhenius, ArrheniusEP

# Load the Mechanism Uncertainty Information

In [2]:
# Make sure you've loaded:
# RMG-Py/save_sensitivity (now you have to rebuild because you turned word-wrap off in chemkin)
# RMG-database/bm_fit_sensitivity

# Load an example ethane model generated using the trees
from rmgpy.tools.uncertainty import Uncertainty, get_rmg_rxn_from_yaml_rxn

chem = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/chem_annotated.inp'
spec = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/species_dictionary.txt'
uncertainty = Uncertainty(output_directory='./temp/uncertainty')
uncertainty.load_model(chem, spec)

In [3]:
# Load the database used to generate the model
uncertainty.load_database(
    thermo_libraries=['BurkeH2O2', 'CurranPentane','FFCM1(-)','primaryThermoLibrary', 'thermo_DFT_CCSDTF12_BAC', 'DFT_QCI_thermo', 'CBS_QB3_1dHR'],
    kinetics_families='default',
    reaction_libraries=['CurranPentane','FFCM1(-)','combustion_core/version5'],
    kinetics_depositories=['training'],
)


In [4]:
# Get the different reaction (and thermo- don't forget thermo) sources
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties()

In [5]:
import json
def unpack_sensitivity(long_desc):
    start_str = 'sensitivities = '
    if start_str not in long_desc:
        return []
    start_index = long_desc.find(start_str) + len(start_str)
    sensitivities_str = long_desc[start_index:].replace("'", '"')
    sensitivities_str = sensitivities_str.replace("nan", '"-9999999"')
    sensitivities_str = sensitivities_str.replace('name', 'training_rxn_name')
    return json.loads(sensitivities_str)

In [6]:
# List the reactions that came from nodes on the new trees and collect the families
print('These reactions were estimated using the new trees:')

families_to_lookup = set()

empty_derivatives = 0
for reaction_key in uncertainty.reaction_sources_dict.keys():
    if 'Rate Rules' in uncertainty.reaction_sources_dict[reaction_key].keys():
        family = reaction_key.family
        src = uncertainty.reaction_sources_dict[reaction_key]['Rate Rules'][1]
        if src['node'] != '':
            # look up the node name in the database
            if src['node'] not in uncertainty.database.kinetics.families[family].rules.entries.keys():
                print('NODE NOT FOUND\t', src['node'])
                continue
            
            entry = uncertainty.database.kinetics.families[family].rules.entries[src['node']][0]
            derivatives = unpack_sensitivity(entry.long_desc)
            if derivatives == []:
                # It's okay- I checked and I just haven't generated the derivatives for those families yet
                continue
            
            print(f'{reaction_key}\t({family}) \n\t{src["node"]}')
            print()
            
            # collect the set of families to lookup
            families_to_lookup.add(str(family))


print(families_to_lookup)


These reactions were estimated using the new trees:
C2H5(44) + C5H11(421) <=> C7H16(1)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R

NC3H7(93) + PC4H9(183) <=> C7H16(1)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R

H(14) + C7H15(682) <=> C7H16(1)	(R_Recombination) 
	Root_1R->H_N-2R->S_N-2CHNO->H_N-2CNO-inRing_Ext-2CNO-R_N-Sp-3R!H=2CCNNOO_N-2CNO->O_Ext-2CN-R

C2H5(44) + C7H15(682) <=> C2H4(11) + C7H16(1)	(Disproportionation) 
	Root_N-4R->H_4CNOS-u1_N-1R!H->O_N-4CNOS->O_Ext-4CNS-R_N-Sp-5R!H#4CCCNNNSSS_N-2R!H->S_N-5R!H->O_Sp-5CS-4CCNSS_Ext-4CNS-R

NC3H7(93) + C7H15(682) <=> C3H6(12) + C7H16(1)	(Disproportionation) 
	Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C

O2(2) + C7H15(682) <=> S(763)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_1COS->O_Ext-1O-R_E

	Root_Ext-2R!H-R_2R!H->C_N-4R->C

C5H11(421) + S(782) <=> C5H10(424) + S(785)	(Disproportionation) 
	Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C

C5H11(422) + S(782) <=> C5H10(424) + S(785)	(Disproportionation) 
	Root_Ext-2R!H-R_2R!H->C_4R->C

C5H11(421) + S(781) <=> C5H10(424) + S(785)	(Disproportionation) 
	Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C

C5H11(422) + S(781) <=> C5H10(424) + S(785)	(Disproportionation) 
	Root_Ext-2R!H-R_2R!H->C_4R->C

C5H11(421) + S(816) <=> C5H10(424) + S(837)	(Disproportionation) 
	Root_Ext-1R!H-R_4R->O_Ext-4O-R_Sp-5R!H-1R!H_Ext-5R!H-R

C5H11(422) + S(816) <=> C5H10(424) + S(837)	(Disproportionation) 
	Root_Ext-2R!H-R_2R!H->C_N-4R->C

C5H11(421) + S(835) <=> C5H10(424) + S(837)	(Disproportionation) 
	Root_Ext-1R!H-R_

In [7]:
print('# Node\tTraining Reaction\t\tdA_node/dA_train\tdE0_node/dA_train\tdn_node/dA_train')
for family in families_to_lookup:
    templateRxnMap = uncertainty.database.kinetics.families[family].get_reaction_matches(thermo_database=uncertainty.database.thermo, remove_degeneracy=True,
                                                                             get_reverse=True, exact_matches_only=False, fix_labels=True)
    
    for r, mechanism_reaction in enumerate(uncertainty.reaction_sources_dict.keys()):
        if mechanism_reaction.__class__.__name__ == 'PDepReaction':
            continue
        if mechanism_reaction.family != family:
            continue
            
        if 'Rate Rules' in uncertainty.reaction_sources_dict[mechanism_reaction].keys():
            # family = reaction_key.family
            src = uncertainty.reaction_sources_dict[mechanism_reaction]['Rate Rules'][1]
            node_name = src['node']
            if node_name != '':
                print(f'{mechanism_reaction.index}\t {mechanism_reaction} \n{node_name}')
                # look up the node name in the database
                if node_name not in uncertainty.database.kinetics.families[family].rules.entries.keys():
                    print('DERIVATIVE NOT YET COMPUTED\t',src['node'])
                    continue
            
                entry = uncertainty.database.kinetics.families[family].rules.entries[node_name][0]
                derivatives = unpack_sensitivity(entry.long_desc)
                
                for i, rxn in enumerate(templateRxnMap[node_name]):
                    N = 35
                    pad_length = N - len(str(rxn))
                    rxn_str = str(rxn) + " " * pad_length
                    #compare names from both sources to check order
                    #print("\n\t", derivatives[i]['name'],"\n\t", rxn_str)
                    dA = derivatives[i]['dA']
                    dE0 = derivatives[i]['dE0']
                    dn = derivatives[i]['dn']
                    if (dn == "-9999999"):
                        dn = np.nan
                    print('\t', rxn_str, "\t\t{0:.6f}".format(dA),
                          "\t\t{0:.6f}".format(dE0),
                          "\t\t{0:.6f}".format(dn))
                    print()
            else:
                print('No node for reaction ', mechanism_reaction) # this needs to complain more loudly
            
    
    
    

# Node	Training Reaction		dA_node/dA_train	dE0_node/dA_train	dn_node/dA_train


ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([82.3338,112.396,135.949,158.789,193.294,214.244,237.75],'J/(mol*K)'), H298=(97.8479,'kJ/mol'), S298=(273.009,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(232.805,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(N3s-NCH) + group(N3d-N3sCd) + group(N3d-CC) + group(CO-NN3dOd) + group(Cd-N3dCdH) + group(Cd-N3dCdH) + ring(Cyclohexane)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([82.3338,112.396,135.949,158.789,193.294,214.244,237.75],'J/(mol*K)'), H298=(97.8479,'kJ/mol'), S298=(273.009,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(232.805,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(N3s-NCH) + group(N3d-N3sCd) + group(N3d-CC) + group(CO-NN3dOd) + group(Cd-N3dCdH) + group(Cd-N3dCdH) + ring(Cyclohexane)""").
The thermo for this species is probably wr

616	 C7H14(808) <=> C7H14(754) 
Root_N-1R!H-inRing
	 C2H4O <=> C2H4O-2                   		-0.000090 		-0.000000 		0.000002

	 C4H8O <=> C4H8O-2                   		-0.000089 		-0.000000 		0.000002

	 C2H4O <=> C2H4O-2                   		0.000000 		-0.000000 		0.000000

	 CH2OS <=> CH2OS-2                   		0.003895 		-0.000000 		0.000002

	 C2H4OS <=> C2H4OS-2                 		0.003896 		-0.000000 		0.000002

	 C3H6OS <=> C3H6OS-2                 		0.003894 		-0.000000 		0.000002

	 C3H6O <=> C3H6O-2                   		0.000000 		-0.000000 		0.000000

	 C2H4N2O2 <=> C2H4N2O2-2             		0.022801 		-0.000000 		0.000002

	 C2H4N2O2-3 <=> C2H4N2O2-4           		0.022788 		-0.000000 		0.000002

	 C2H4N2O2-5 <=> C2H4N2O2-6           		0.022780 		-0.000000 		0.000002

	 C2H4N2O2-7 <=> C2H4N2O2-8           		0.022798 		-0.000000 		0.000002

	 C3H3NO <=> C3H3NO-2                 		0.022775 		-0.000000 		0.000002

	 C3H6N2O <=> C3H6N2O-2               		0.022780 		-0.000000 		0.000002

	 C2H3NO2-3 <=> C2H3NO2-4             		0.022779 		-0.000000 		0.000002

	 C2H3NO2-3 <=> C2H3NO2-4             		0.022786 		-0.000000 		0.000002

	 C2H4N2O-3 <=> C2H4N2O-4             		0.022836 		-0.000000 		0.000002

	 C2H5N3O-3 <=> C2H5N3O-4             		0.022791 		-0.000000 		0.000002

	 C2H4N2O-5 <=> C2H4N2O-6             		0.022781 		-0.000000 		0.000002

	 CC=O <=> C=CO                       		-0.000089 		-0.000000 		0.000002

	 CCC(C)=O <=> C=C(O)CC               		-0.000089 		-0.000000 		0.000002

	 CC=O <=> C=CO                       		-0.000089 		-0.000000 		0.000002

	 SC=O <=> OC=S                       		0.003895 		-0.000000 		0.000002

	 CC(=O)S <=> CC(=S)O                 		0.003895 		-0.000000 		0.000002

	 CCC(=O)S <=> CCC(=S)O               		0.003895 		-0.000000 		0.000002

	 CCC=O <=> C=COC                     		-0.000088 		-0.000000 		0.000002

	 O=CNC(=O)N <=> NC(=NC=O)O           		0.022802 		-0.000000 		0.000002

	 O=CNC(=O)N <=> OC(=N)NC=O           		0.02278

	 C2H4N2O2 <=> C2H4N2O2-2             		0.022801 		-0.000000 		0.000002

	 C2H4N2O2-3 <=> C2H4N2O2-4           		0.022788 		-0.000000 		0.000002

	 C2H4N2O2-5 <=> C2H4N2O2-6           		0.022780 		-0.000000 		0.000002

	 C2H4N2O2-7 <=> C2H4N2O2-8           		0.022798 		-0.000000 		0.000002

	 C3H3NO <=> C3H3NO-2                 		0.022775 		-0.000000 		0.000002

	 C3H6N2O <=> C3H6N2O-2               		0.022780 		-0.000000 		0.000002

	 C2H5N3O <=> C2H5N3O-2               		0.022783 		-0.000000 		0.000002

	 C2H3NO2 <=> C2H3NO2-2               		0.022794 		-0.000000 		0.000002

	 C2H4N2O2-9 <=> C2H4N2O2-10          		0.022787 		-0.000000 		0.000002

	 C2H4N2O <=> C2H4N2O-2               		0.022788 		-0.000000 		0.000002

	 CH4N2O <=> CH4N2O-2                 		0.022785 		-0.000000 		0.000002

	 C2H4N2O2-11 <=> C2H4N2O2-12         		0.022787 		-0.000000 		0.000002

	 C2H4N2O2-13 <=> C2H4N2O2-14         		0.022781 		-0.000000 		0.000002

	 C2H3NO2-3 <=> C2H3NO2-4             		0.022779 		

...

	 HO2-2 + H2NO <=> H2O2 + HNO-2       		0.199866 		0.000000 		0.000003

	 HO2-2 + CH2N <=> H2O2 + CHN         		0.199440 		0.000000 		0.000038

2043	 C2H5O2(45) + NC3H7(93) <=> C2H6O2(46) + C3H6(12) 
Root_Ext-1R!H-R_4R->O_Ext-4O-R_Sp-5R!H-1R!H
	 C3H7-2 + O2 <=> HO2 + C3H6-2        		-3.547821 		-0.089822 		-0.559650

	 C4H9-2 + O2 <=> HO2 + C4H8-2        		-1.826847 		-0.047731 		-0.297412

	 C4H7-3 + O2 <=> HO2 + C4H6-2        		0.676709 		0.013498 		0.084074

	 C3H7-2 + O2 <=> HO2 + C3H6-2        		-3.546455 		-0.089789 		-0.559441

	 C4H7-3 + O2 <=> HO2 + C4H6-2        		0.681243 		0.013599 		0.084772

	 C4H9-2 + O2 <=> HO2 + C4H8-2        		-1.830373 		-0.047809 		-0.297956

	 C5H9-2 + O2 <=> HO2 + C5H8-3        		2.466544 		0.057263 		0.356811

	 C6H9 + O2 <=> HO2 + C6H8            		2.750491 		0.064205 		0.400080

2044	 C2H5O2(45) + IC3H7(94) <=> C2H6O2(46) + C3H6(12) 
Root_Ext-2R!H-R_2R!H->C_N-4R->C
	 C4H7-2 + O2 <=> HO2 + C4H6          		0.999500 		0.000000 		nan

2045	 CH3O2(4

	 C3H5 + C3H7-2 <=> C3H6 + C3H6-2     		0.004281 		-0.000000 		-0.000162

	 C2H3S + C4H9-2 <=> C2H4S + C4H8-2   		0.004280 		-0.000000 		-0.000162

	 CH3O-2 + C3H7-2 <=> CH4O + C3H6-2   		0.004280 		-0.000000 		-0.000162

	 C4H9-2 + CH3S <=> CH4S + C4H8-2     		0.004280 		-0.000000 		-0.000162

	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2     		0.004279 		-0.000000 		-0.000162

	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2    		0.004279 		-0.000000 		-0.000162

	 C2H3-2 + C3H7-2 <=> C2H4-2 + C3H6-2 		0.004279 		-0.000000 		-0.000162

	 HS2 + C5H9 <=> H2S2 + C5H8-2        		0.004276 		-0.000000 		-0.000161

	 C2H + C3H7-2 <=> C2H2 + C3H6-2      		0.004285 		-0.000000 		-0.000163

	 HO + C3H7-2 <=> H2O + C3H6-2        		0.004275 		-0.000000 		-0.000161

	 C4H9-2 + O2 <=> HO2 + C4H8-2        		0.004275 		-0.000000 		-0.000161

	 C2H + C4H9-2 <=> C2H2 + C4H8-2      		0.004285 		-0.000000 		-0.000163

	 H + C4H9-2 <=> H2 + C4H8-2          		0.004280 		-0.000000 		-0.000162

	 CH3_r1 + C4H9-2 <=> CH4 + C4H8-2    

	 C4H9-2 + C2H5-2 <=> C2H6 + C4H8-2   		0.004282 		-0.000000 		-0.000162

	 CH3O-2 + C4H9-2 <=> CH4O + C4H8-2   		0.004280 		-0.000000 		-0.000162

	 C3H5 + C4H9-2 <=> C3H6 + C4H8-2     		0.004278 		-0.000000 		-0.000161

	 C4H9-2 + C3H7 <=> C3H8 + C4H8-2     		0.004278 		-0.000000 		-0.000162

	 C4H9-2 + C4H9 <=> C4H10 + C4H8-2    		0.004280 		-0.000000 		-0.000162

	 C2H3-2 + C4H9-2 <=> C2H4-2 + C4H8-2 		0.004282 		-0.000000 		-0.000162

	 HO + C4H9-2 <=> H2O + C4H8-2        		0.004283 		-0.000000 		-0.000163

	 C3H5-2 + O2 <=> HO2 + C3H4          		0.004278 		-0.000000 		-0.000162

	 CH3_r1 + C3H5-2 <=> CH4 + C3H4      		0.004279 		-0.000000 		-0.000162

	 C3H5-2 + C2H5-2 <=> C2H6 + C3H4     		0.004276 		-0.000000 		-0.000161

	 C3H5 + C3H5-2 <=> C3H6 + C3H4       		0.004276 		-0.000000 		-0.000161

	 C3H5-2 + C3H7 <=> C3H8 + C3H4       		0.004277 		-0.000000 		-0.000161

	 C3H5-2 + C4H9 <=> C4H10 + C3H4      		0.004282 		-0.000000 		-0.000163

	 C2H3-2 + C3H5-2 <=> C2H4-2 + C3H4   

	 C3H7-2 + C2H5-2 <=> C2H6 + C3H6-2   		0.004284 		-0.000000 		-0.000163

	 C5H7 + C2H5S <=> C5H8 + C2H4S-2     		0.004281 		-0.000000 		-0.000162

	 C3H5 + C3H7-2 <=> C3H6 + C3H6-2     		0.004281 		-0.000000 		-0.000162

	 C2H3S + C4H9-2 <=> C2H4S + C4H8-2   		0.004280 		-0.000000 		-0.000162

	 CH3O-2 + C3H7-2 <=> CH4O + C3H6-2   		0.004280 		-0.000000 		-0.000162

	 C4H9-2 + CH3S <=> CH4S + C4H8-2     		0.004280 		-0.000000 		-0.000162

	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2     		0.004279 		-0.000000 		-0.000162

	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2    		0.004279 		-0.000000 		-0.000162

	 C2H3-2 + C3H7-2 <=> C2H4-2 + C3H6-2 		0.004279 		-0.000000 		-0.000162

	 HS2 + C5H9 <=> H2S2 + C5H8-2        		0.004276 		-0.000000 		-0.000161

	 C2H + C3H7-2 <=> C2H2 + C3H6-2      		0.004285 		-0.000000 		-0.000163

	 HO + C3H7-2 <=> H2O + C3H6-2        		0.004275 		-0.000000 		-0.000161

	 C4H9-2 + O2 <=> HO2 + C4H8-2        		0.004275 		-0.000000 		-0.000161

	 C2H + C4H9-2 <=> C2H2 + C4H8-2      

	 C4H7 + C2H5S <=> C4H8 + C2H4S-2     		0.997054 		0.180872 		0.261970

	 C5H7 + C2H5S <=> C5H8 + C2H4S-2     		-1.982106 		-0.541049 		-0.782565

	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2     		0.996762 		0.180869 		0.261859

	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2    		0.996554 		0.180863 		0.261780

2172	 SC4H9(184) + S(1342) <=> C4H8(190) + C7H14(808) 
Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C
	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2     		0.499732 		0.000000 		-0.000005

	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2    		0.499784 		0.000000 		0.000009

2173	 SC4H9(184) + S(1342) <=> C4H8(190) + C7H14(754) 
Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C
	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2     		0.499732 		0.000000 		-0.000005

	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2    		0.499784 		0.

# Save the Mechanism Uncertainty Information

In [8]:
import yaml
import cantera as ct
import pickle
import numpy as np

In [9]:
# see if we can use the reaction as the dictionary key
mechanism_yaml = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/nheptane.yaml'
gas1 = ct.Solution(mechanism_yaml)
len(gas1.reactions())

2266

In [10]:
# Read in the yaml mechanism file
with open(mechanism_yaml, 'r') as file:
    mech = yaml.load(file, Loader=yaml.FullLoader)
    # print(mech)

In [11]:
auto_gen_families = {}
for family_name in uncertainty.database.kinetics.families.keys():
    if uncertainty.database.kinetics.families[family_name].auto_generated and family_name not in auto_gen_families.keys():
        auto_gen_families[family_name] = uncertainty.database.kinetics.families[family_name].rules.get_entries()
        auto_gen_families[f'{family_name}_labels'] = [entry.label for entry in uncertainty.database.kinetics.families[family_name].rules.get_entries()]
        auto_gen_families[f'{family_name}_rxn_map'] = uncertainty.database.kinetics.families[family_name].get_reaction_matches(thermo_database=uncertainty.database.thermo, remove_degeneracy=True,
                                                                             get_reverse=True, exact_matches_only=False, fix_labels=True)
# print(auto_gen_families)
#if uncertainty.database.kinetics.families[family_name].auto_generated:

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([82.3338,112.396,135.949,158.789,193.294,214.244,237.75],'J/(mol*K)'), H298=(97.8479,'kJ/mol'), S298=(273.009,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(232.805,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(N3s-NCH) + group(N3d-N3sCd) + group(N3d-CC) + group(CO-NN3dOd) + group(Cd-N3dCdH) + group(Cd-N3dCdH) + ring(Cyclohexane)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([82.3338,112.396,135.949,158.789,193.294,214.244,237.75],'J/(mol*K)'), H298=(97.8479,'kJ/mol'), S298=(273.009,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(232.805,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(N3s-NCH) + group(N3d-N3sCd) + group(N3d-CC) + group(CO-NN3dOd) + group(Cd-N3dCdH) + group(Cd-N3dCdH) + ring(Cyclohexane)""").
The thermo for this species is probably wr

In [170]:
def get_kinetics_dict(data):
    kinetics_dict = {}
    if isinstance(data, Arrhenius):
        ct_data = data.to_cantera_kinetics()
        kinetics_dict['A'] = ct_data.pre_exponential_factor
        kinetics_dict['Ea'] = ct_data.activation_energy
        kinetics_dict['n'] = ct_data.temperature_exponent
    elif isinstance(data, ArrheniusEP):
        kinetics_dict['A'] = data.A.value_si
        kinetics_dict['E0'] = data.E0.value_si
        kinetics_dict['n'] = data.n.value_si
        kinetics_dict['alpha'] = data.alpha.value_si
    else:
        print('Reaction type not yet implement', type(data))
    return kinetics_dict

In [171]:
# Fill out the uncertainty yaml kinetics

for i, reaction in enumerate(mech['reactions']):
    rmg_rxn = get_rmg_rxn_from_yaml_rxn(uncertainty.reaction_list, reaction)
    # print(uncertainty.reaction_sources_dict[rmg_rxn])
    
    rxn_uncertainty = {}
    rxn_uncertainty['value'] = float(uncertainty.kinetic_input_uncertainties[uncertainty.reaction_list.index(rmg_rxn)])
    # Note the source- assume only one source per reaction, or else this might overwrite existing reaction
    for src_key in uncertainty.reaction_sources_dict[rmg_rxn].keys():
        rxn_uncertainty['source'] = src_key
    if src_key == 'Rate Rules':
        family_name = str(rmg_rxn.family)
        rxn_uncertainty['family'] = family_name
        src = uncertainty.reaction_sources_dict[rmg_rxn]['Rate Rules'][1]
        new_trees = (src['node'] != '')
        if new_trees:
            rxn_uncertainty['rule_node'] = src['node']
            rule_index = auto_gen_families[family_name + '_labels'].index(src['node'])
            rule = auto_gen_families[family_name][rule_index]
            derivatives = unpack_sensitivity(rule.long_desc)
            
            
            # check that the sensitivities correspond to the correct training reactions
            if len(auto_gen_families[family_name + '_rxn_map'][src['node']]) != len(derivatives):
                print('mismatch', src['node'], family_name)
                
            rxn_uncertainty['sensitivities'] = derivatives
            for j in range(0, len(derivatives)):
                try:
                    t_rxn = auto_gen_families[family_name + '_rxn_map'][src['node']][j]
                    ct_t_rxn = t_rxn.to_cantera()
                except:
                    print(family_name, auto_gen_families[family_name + '_rxn_map'][src['node']][j])
                    continue
                derivatives[j]['training_name'] = ct_t_rxn.equation
                derivatives[j]['training_rate'] = get_kinetics_dict(t_rxn.kinetics)
            # match the training reaction to the actual object
        # handle old trees
        else:
            # check for an exact match
            uncertainties = []
            info = uncertainty.reaction_sources_dict[rmg_rxn]['Rate Rules']

            family_name = info[0]
            rxn_uncertainty['family'] = family_name
            rxn_uncertainty['exact'] = info[1]['exact']

            for rule_entry, train_entry, weight in info[1]['training']:
                rate = train_entry.data.to_cantera_kinetics()
                uncertainties.append({'training_name': train_entry.label,
                                      'training_rate': get_kinetics_dict(train_entry.data),
                                      'weight': weight,
                                      'training_desc': train_entry.long_desc})
            for rule_entry, weight in info[1]['rules']:
                uncertainties.append({'rule_name': rule_entry.label,
                                      'rule_rate': get_kinetics_dict(rule_entry.data),
                                      'weight': weight,
                                      'rule_desc': rule_entry.long_desc})
            rxn_uncertainty['train_rxn_uncertainty'] = uncertainties
                
    
    mech['reactions'][i]['uncertainty'] = rxn_uncertainty
    #if i > 500:
    #    break

In [204]:
# Fill out the uncertainty yaml thermo

#print(uncertainty.)
for i, species in enumerate(mech['species']):
    species_uncertainty = {}
    uncertainty_entry = uncertainty.species_sources_dict[uncertainty.species_list[i]]
    #print(uncertainty_entry)
    if 'Library' in uncertainty_entry.keys():
        species_uncertainty['source'] = 'Library'
    elif 'GAV' in uncertainty_entry.keys():
        group_entry_uncertainties = []
        species_uncertainty['source'] = 'GAV'
        for key in uncertainty_entry['GAV'].keys():
            for entry, freq in uncertainty_entry['GAV'][key]:
                group_entry_uncertainties.append({'family_name': key,
                                                  'entry_name': entry.label,
                                                  'entry_index': entry.index,
                                                  'entry_freq': freq})
        species_uncertainty['group_entries'] = group_entry_uncertainties
        
    else:
        print('Missing species uncertainty source')
    
    species_uncertainty['value'] = float(uncertainty.thermo_input_uncertainties[i])

    mech['species'][i]['uncertainty'] = species_uncertainty

#print(uncertainty.species_list[0].label)

#print(len(mech['species']))
#print(len(uncertainty.species_list))
#for i, species in enumerate(mech['species']):
#    if species['name'] != str(uncertainty.species_list[i]):
#        print(species['name'], uncertainty.species_list[i].label)

#print(uncertainty.species_list)

In [205]:
output_path = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_uncertainty.yaml'
with open(output_path, 'w') as file:
    output = yaml.dump(mech, file)
    print(output)

None


In [ ]:
# check for duplicate names - yes there are duplicate equations
reaction_names = []
reaction_IDs = []
for reaction_key in uncertainty.reaction_sources_dict.keys():
    try:
        out_reaction = reaction_key.to_cantera()
        if type(out_reaction) is list:
            out_reaction = out_reaction[0]
        if out_reaction.equation in reaction_names:
            print(f'Duplicate: {out_reaction.equation}')
        reaction_names.append(out_reaction.equation)
        reaction_IDs.append(out_reaction.ID)
    except AttributeError:
        print(f'Failed to convert {reaction_key}')
        print(reaction_key.kinetics)
        # raise
print('\nReaction equations')
print(len(reaction_names))
print(len(set(reaction_names)))
print('\nReaction IDs')
print(len(reaction_IDs))
print(len(set(reaction_IDs)))

In [ ]:
print(len(uncertainty.kinetic_input_uncertainties))
print(len(uncertainty.reaction_sources_dict.keys()))

[{'name': 'N2', 'composition': {'N': 2}, 'thermo': {'model': 'NASA7', 'temperature-ranges': [100.0, 1817.05, 5000.0], 'data': [[3.61262543, -0.00100892648, 2.4989762e-06, -1.43374896e-09, 2.58633809e-13, -1051.10286, 2.65270528], [2.97592894, 0.0016413738, -7.19704298e-07, 1.25374055e-10, -7.91499564e-15, -1025.85898, 5.53740798]], 'note': 'Thermo library: BurkeH2O2'}, 'transport': {'model': 'gas', 'geometry': 'linear', 'well-depth': 97.53, 'diameter': 3.621, 'polarizability': 1.76, 'rotational-relaxation': 4.0, 'note': 'PrimaryTransportLibrary'}, 'note': 'N2'}, {'name': 'Ar', 'composition': {'Ar': 1}, 'thermo': {'model': 'NASA7', 'temperature-ranges': [100.0, 4762.74, 5000.0], 'data': [[2.5, -1.87769028e-11, 2.35957382e-14, -9.62803665e-18, 1.2072768e-21, -745.0, 4.36630362], [2.87626762, -0.000312580154, 9.73654773e-08, -1.34776059e-11, 6.99515521e-16, -1107.30235, 1.95965953]], 'note': 'Thermo library: BurkeH2O2'}, 'transport': {'model': 'gas', 'geometry': 'atom', 'well-depth': 136.

In [186]:
dir(uncertainty)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'all_kinetic_sources',
 'all_thermo_sources',
 'assign_parameter_uncertainties',
 'compile_all_sources',
 'database',
 'extra_species',
 'extract_sources_from_model',
 'kinetic_input_uncertainties',
 'load_database',
 'load_model',
 'local_analysis',
 'output_directory',
 'reaction_list',
 'reaction_sources_dict',
 'retrieve_saturated_species_from_list',
 'sensitivity_analysis',
 'species_list',
 'species_sources_dict',
 'thermo_input_uncertainties']

{Species(label="N2", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.61263,-0.00100893,2.49898e-06,-1.43375e-09,2.58634e-13,-1051.1,2.65271], Tmin=(100,'K'), Tmax=(1817.05,'K')), NASAPolynomial(coeffs=[2.97593,0.00164137,-7.19704e-07,1.25374e-10,-7.915e-15,-1025.86,5.53741], Tmin=(1817.05,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: BurkeH2O2"""), molecule=[Molecule(smiles="N#N")], reactive=False, molecular_weight=(28.0137,'amu')): {'Library': 0},
 Species(label="Ar", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[2.5,-1.87769e-11,2.35957e-14,-9.62804e-18,1.20728e-21,-745,4.3663], Tmin=(100,'K'), Tmax=(4762.74,'K')), NASAPolynomial(coeffs=[2.87627,-0.00031258,9.73655e-08,-1.34776e-11,6.99516e-16,-1107.3,1.95966], Tmin=(4762.74,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: BurkeH2O2"""), molecule=[Molecule(smiles="[Ar]")], molecular_weight=(39.8775,'amu')): {'Library': 1},
 Species(label="Ne", thermo=NASA(

In [200]:
uncertainty_entry['GAV']

{'group': [(<Entry index=2312 label="O2s-OsCs">, 1),
  (<Entry index=2294 label="O2s-OsH">, 1),
  (<Entry index=1892 label="Cs-(Cds-Cds)CsOsH">, 1),
  (<Entry index=933 label="Cs-CsCsHH">, 2),
  (<Entry index=901 label="Cs-CsHHH">, 1),
  (<Entry index=905 label="Cs-(Cds-Cds)HHH">, 1),
  (<Entry index=325 label="Cds-CdsCsH">, 2)],
 'radical': [(<Entry index=1031 label="RCCJC">, 1)]}